# **US DEBT TO GDP**
---
- Tracking Total Debt of Economy as a Percentage of the Total Income of the Economy (GDP)
- Tracking Total Amount of Debt Service Payments Relative to GDP
- Over 12yr Period


### Import Libraries 

In [4]:
import os
import csv

import pandas as pd 
import numpy as np 
import datetime as dt 

import matplotlib.pyplot as plt

### Import US GDP and Debt from [Federal Reserve Economic Data Repository (FRED)](https://fred.stlouisfed.org/)

In [5]:
files = os.listdir('data/FRED')
measurements = {}
for file in files: 
    if file.split('.')[1] == 'csv':
        name = file.split('.')[0]
        measurements[name] = pd.read_csv('data/FRED/'+file, index_col='DATE')
        measurements[name].index = pd.to_datetime(measurements[name].index)
print('List of Measurements: ', end=' ')
for i in measurements.keys(): 
    print(i.upper(), end=' ')
print('\n')
print(pd.read_csv('data/references/FREDdata_refs.csv'))

List of Measurements:  ASTLL ASTDSL ASCMA ASFMA NCBDBIQ027S GFDEBTN MTSO133FMS NCBLL FGDSLAQ027S REVOLSL ASMRMA SLGSDODNS ASHMA MVLOAS SLOAS FBLL NNBLL GDP TOTALSL FBDSILQ027S 

      Series_ID                                            URLS
0         ASCMA        https://fred.stlouisfed.org/series/ASCMA
1         ASFMA        https://fred.stlouisfed.org/series/ASFMA
2         ASHMA        https://fred.stlouisfed.org/series/ASHMA
3        ASMRMA       https://fred.stlouisfed.org/series/ASMRMA
4        ASTDSL       https://fred.stlouisfed.org/series/ASTDSL
5         ASTLL        https://fred.stlouisfed.org/series/ASTLL
6   FBDSILQ027S  https://fred.stlouisfed.org/series/FBDSILQ027S
7          FBLL         https://fred.stlouisfed.org/series/FBLL
8   FGDSLAQ027S  https://fred.stlouisfed.org/series/FGDSLAQ027S
9           GDP          https://fred.stlouisfed.org/series/GDP
10      GFDEBTN      https://fred.stlouisfed.org/series/GFDEBTN
11       MVLOAS       https://fred.stlouisfed.org/seri

In [ ]:
def _1960_Present(file_addy): 
    new_df = file_addy['1960-01-01':'2021-10-01']
    new_df = new_df.astype(float, errors='raise')
    #new_df.info()
    return new_df

### Find Total Debt 

In [ ]:
# All Sectors; Total Loans; Liability, Level 
ASTLL = _1960_Present(measurements['ASTLL'])
# All Sectors; Total Debt Securities; Liability, Level
ASTDSL = _1960_Present(measurements['ASTDSL'])
# Gross Domestic Product 
GDP = _1960_Present(measurements['GDP'])

In [ ]:
total_debt = pd.concat([ASTLL,ASTDSL,GDP], axis=1, join='outer')
total_debt.info()
total_debt.isnull().sum()

In [ ]:
# GDP measured in BILLIONS 
# ASTLL + ASTDSL measured in MILLIONS
total_debt['GDP'] = total_debt['GDP'] * 1000

In [ ]:
total_debt['Total_Debt'] = total_debt['ASTLL'] + total_debt['ASTDSL']
total_debt['Total_Debt_%GDP'] = total_debt['Total_Debt']/total_debt['GDP']

total_debt.tail()

In [ ]:
plt.figure(figsize=(16,7))

plt.plot(total_debt['Total_Debt_%GDP'], label='Total Debt %GDP')

plt.title('Total Debt of US as a Percentage of GDP', fontsize=15)
plt.ylabel('Percent of Total Debt to GDP', fontsize=12)
plt.legend(loc=2)
plt.show()

## By Category 

In [11]:
import debtToGDP as dtg # made into class now-> calling functions gunna be whacky 

# desired date range 
start = '1960-01-01'
end = '2021-10-01'

In [12]:
debtion = dtg.DEBT(measurements,start,end)

debtion.total_household_nonprofit_debt()

TypeError: date_range() takes 2 positional arguments but 4 were given

In [ ]:
# Business

def total_business_debt(start,end): 
    tot_commercial_mort = dtg.date_range(dtg.measurements['ASCMA'],start,end)
    tot_farm_mort = dtg.date_range(dtg.measurements['ASFMA'],start,end)
    tot_fin_biz_debt_secur = dtg.date_range(dtg.measurements['FBDSILQ027S'],start,end)
    tot_fin_biz_loans_liab = dtg.date_range(dtg.measurements['FBLL'],start,end)
    tot_nf_corp_debt_secur = dtg.date_range(dtg.measurements['NCBDBIQ027S'],start,end)
    tot_nf_corp_loans = dtg.date_range(dtg.measurements['NCBLL'],start,end)
    tot_nf_ncorp_biz_loans = dtg.date_range(dtg.measurements['NNBLL'],start,end)
    
    tot_biz_debt = pd.concat([tot_commercial_mort, tot_farm_mort, tot_fin_biz_debt_secur,
                             tot_fin_biz_loans_liab, tot_nf_corp_debt_secur, tot_nf_corp_loans,
                             tot_nf_ncorp_biz_loans],
                             axis=1, join='outer')
    return tot_biz_debt


In [ ]:
biz_debt = total_business_debt(start,end)
#biz_debt.info()
#biz_debt.isnull().sum()
biz_debt

In [ ]:
# Government: 

def total_gov_debt(start,end): 
    fed_securities = dtg.date_range(measurements['FGDSLAQ027S'],start,end) #millions Quarterly
    state_local = dtg.date_range(measurements['SLGSDODNS'],start,end)*1000 #billions Quarterly
    #fed_non_securities = dtg.date_range(measurements['GFDEBTN-NONSEC'])
    
    tot_gov_debt = pd.concat([fed_securities, state_local],
                            axis=1, join='outer')
    
    tot_gov_debt = tot_gov_debt.fillna(0)
    #tot_gov_debt = tot_gov_debt[tot_gov_debt[] != 0] #no non-quarterly data
    tot_gov_debt['debt_sum'] = tot_gov_debt.sum(axis=1)
    return tot_gov_debt

In [ ]:
gov_debt = total_gov_debt(start,end)
gov_debt.tail(20)

In [ ]:
# Household + Non-Profit 
def total_household_nonprofit_debt(start,end): 
    onefour_mort = dtg.date_range(dtg.measurements['ASHMA'],start,end) #millions Quarterly
    multi_mort = dtg.date_range(dtg.measurements['ASMRMA'],start,end) #millions Quarterly
    auto_loan = dtg.date_range(dtg.measurements['MVLOAS'],start,end)*1000 #billions Quarterly
    stu_loan = dtg.date_range(dtg.measurements['SLOAS'],start,end)*1000 #billions Quarterly
    credit_card = dtg.date_range(dtg.measurements['REVOLSL'],start,end)*1000 #billions MONTHLY
    
    tot_house_debt = pd.concat([onefour_mort, multi_mort, auto_loan, stu_loan, credit_card], 
                               axis=1, join='outer')
    
    tot_house_debt = tot_house_debt.fillna(0)
    tot_house_debt = tot_house_debt[tot_house_debt['ASHMA'] != 0]
    tot_house_debt['debt_sum'] = tot_house_debt.sum(axis=1)

    return tot_house_debt


In [ ]:
house_debt = total_household_nonprofit_debt(start,end)
#house_debt.tail(20)

In [ ]:
plt.figure(figsize=(16,7))

plt.plot(house_debt)

plt.title('Household and Non-Profit Debt', fontsize=15)
#plt.yticks(np.linspace(0,2000000,10000, dtype=int))
plt.ylabel('Debt in Millions', fontsize=12)
plt.legend(loc=2)
plt.show()

# Work In Progress
---

In [ ]:
measurements['SLGSDODNS'].tail(20)

In [ ]:
files = os.listdir('data/FRED')
measurements = {}
for file in files: 
    if file.split('.')[1] == 'csv':
        name = file.split('.')[0]
        measurements[name] = pd.read_csv('data/FRED/'+file, index_col='DATE')
        measurements[name].index = pd.to_datetime(measurements[name].index).to_period('Q').strftime('q%q %Y')
        #quarterly = pd.date_range(start,end, freq='QS')

In [ ]:
measurements['REVOLSL'].tail(15)
#quarterly

In [ ]:
files = os.listdir('data/FRED')
measurements = {}
for file in files: 
    if file.split('.')[1] == 'csv':
        name = file.split('.')[0]
        measurements[name] = pd.read_csv('data/FRED/'+file, index_col='DATE')
        measurements[name].index = pd.DatetimeIndex(measurements[name].index)
        measurements[name].index.quarter

print('List of Measurements: ', end=' ')
for i in measurements.keys(): 
    print(i.upper(), end=' ')
        
measurements['ASHMA'].tail()